In [1]:
library(tidyverse)
library(repr)
library(tidymodels)
library(cowplot)
options(repr.matrix.max.rows = 6)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

In [2]:
PLAYERS_DATA_URL = "https://raw.githubusercontent.com/Michael-R-Dickinson/DSCI-100-individual-project/refs/heads/main/players.csv"

# Download Data
download.file(PLAYERS_DATA_URL, "players.csv")

# Load Data
players_df <- read_csv('players.csv') 

# Clean Data

# Remove NA's and add levels to experience for visualization
players_df <- players_df |>
    filter(!is.na(Age)) |>
    mutate(
        "age" = Age,
        "subscribe" = as.factor(subscribe),
        "hashed_email" = hashedEmail,
        "experience" = factor(
            experience, 
            levels = c("Beginner", "Amateur", "Regular", "Veteran", "Pro"), 
        )
    ) |>
    select(-Age, -hashedEmail)

head(players_df)

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


experience,subscribe,played_hours,name,gender,age,hashed_email
<fct>,<fct>,<dbl>,<chr>,<chr>,<dbl>,<chr>
Pro,TRUE,30.3,Morgan,Male,9,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d
Veteran,TRUE,3.8,Christian,Male,17,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9
Veteran,FALSE,0.0,Blake,Male,17,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28
Amateur,TRUE,0.7,Flora,Female,21,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5
Regular,TRUE,0.1,Kylie,Male,21,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e
Amateur,TRUE,0.0,Adrian,Female,17,f58aad5996a435f16b0284a3b267f973f9af99e7a89bee0430055a44fa92f977


In [ ]:
players_df <- players_df |>
    mutate("experience" = as.numeric(experience))


players_datasets <- initial_split(players_df, prop = 0.8, strata = subscribe)
train_set <- training(players_datasets)
test_set <- testing(players_datasets)

folds <- vfold_cv(data = train_set, v = 5, strata = subscribe)

classification_recipe <- recipe(subscribe ~ played_hours + age + experience, data = players_df) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors())

tune_grid <- tibble(neighbors = seq(from = 1, to = 30, by = 1))
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
    set_engine("kknn") |>
    set_mode("classification")

classification_workflow <- workflow() |>
    add_recipe(classification_recipe) |>
    add_model(knn_spec) |>
    tune_grid(resamples = folds, grid = tune_grid)

metrics <- classification_workflow |> collect_metrics()

accuracies <- metrics |>
    filter(.metric == "accuracy") |>
    select(neighbors, mean)

accuracy_by_neighbors_plot <- accuracies |> ggplot(aes(x = neighbors, y = mean)) +
    geom_line() + 
    geom_point() +
    labs(
        x = "Number of Neighbors", 
        y = "Cross Validation Accuracy",
        title = "Accuracy vs Number of Neighbors"
    ) + 
    theme(text = element_text(size = 20))

optimal_number_of_neighbors <- accuracies |>
    arrange(order_by = desc(mean)) |>
    slice(1) |>
    pull(neighbors)

optimal_knn_spec <-  nearest_neighbor(weight_func = "rectangular", neighbors = optimal_number_of_neighbors) |>
    set_engine("kknn") |>
    set_mode("classification")

optimal_workflow <- workflow() |>
    add_recipe(classification_recipe) |>
    add_model(optimal_knn_spec) |>
    fit(data = train_set)

test_set_metrics <- optimal_workflow |>
    predict(test_set) |>
    bind_cols(test_set) |>
    metrics(truth = subscribe, estimate = .pred_class)

In [ ]:
accuracy_by_neighbors_plot

In [ ]:
test_set_metrics

In [ ]:
predictors_df <- players_df |> 
    select(played_hours, age, experience, subscribe)

In [ ]:
## Analyzing Correlation Between Each Set of Variables
options(repr.plot.width = 10, repr.plot.height = 6) 
predictors_df |>
    ggplot(aes(x = age, y = played_hours, color = subscribe)) + 
    geom_point() + 
    scale_y_log10() + 
    labs(
        x = "Age",
        y = "Hours Played",
        color = "Subscribed to Newsletter",
        title = "Age vs Hours Played"
    ) + 
    theme(text = element_text(size = 15))

options(repr.plot.width = 10, repr.plot.height = 6) 
predictors_df |>
    ggplot(aes(x = experience, y = age, color = subscribe)) +
    geom_point() + 
    labs(
        x = "Experience",
        y = "Age",
        color = "Subscribed to Newsletter",
        title = "Experience vs Age"
    ) + 
    theme(text = element_text(size = 15))

options(repr.plot.width = 10, repr.plot.height = 6) 
predictors_df |>
    ggplot(aes(x = played_hours, y = experience, color = subscribe)) +
    geom_point() + 
    scale_x_log10() + 
    labs(
        x = "Hours Played",
        y = "Experience",
        color = "Subscribed to Newsletter",
        title = "Hours Played vs Experience"
    ) +
    theme(text = element_text(size = 15))

In [ ]:
# Histograms with Subscribe coloring

options(repr.plot.width = 8, repr.plot.height = 6)
predictors_df |> ggplot(aes(x = played_hours, fill = subscribe)) +
    geom_histogram(bins = 12) +
    scale_x_log10() + 
    labs(
            x = "Hours Played",
            y = "Number of Players",
            fill = "Subscribed",
            title = "Played Hours Colored by Subscription"
    ) + 
    theme(text = element_text(size = 15))

options(repr.plot.width = 8, repr.plot.height = 6)
predictors_df |> ggplot(aes(x = age, fill = subscribe)) +
    geom_histogram(bins = 12) +
    labs(
            x = "Player Age",
            y = "Number of Players",
            fill = "Subscribed",
            title = "Player Age Colored by Subscription"
    ) + 
    # Log scaling on y but without dropping all rows that cause 0 values
    # because log(0) = infinity
    scale_y_continuous(trans = scales::pseudo_log_trans()) + 
    theme(text = element_text(size = 15))

options(repr.plot.width = 8, repr.plot.height = 6)
predictors_df |> ggplot(aes(x = experience, fill = subscribe)) +
    geom_histogram(bins = 5) + 
    labs(
            x = "Experience Level",
            y = "Number of Players",
            fill = "Subscribed",
            title = "Player Experience Colored by Subscription"
    ) + 
    theme(text = element_text(size = 15))



In [ ]:
# Bars
# These did not go to plan
predictors_df |> 
    ggplot(aes(x = age, fill = subscribe)) +
    geom_bar(position = "fill") + 
    labs(title = "Don't even worry about this disaster")

ratio_df <- predictors_df |> 
  mutate(bin = cut(age, breaks = 15)) |> 
  group_by(bin) |> 
  summarize(
    subscribed = sum(subscribe == "TRUE"),
    total = n()
  ) |> 
  ungroup() |> 
  mutate(
    subscribe_ratio = subscribed / total,
    not_subscribe_ratio = 1 - subscribe_ratio
  )
ratio_long <- ratio_df |> 
  pivot_longer(
    cols = c(subscribe_ratio, not_subscribe_ratio),
    names_to = "subscription_status",
    values_to = "ratio"
  )
# Same plot as before but trying to make the bins in the x axis smaller
ratio_long |> 
  ggplot(aes(x = bin, y = ratio, fill = subscription_status)) +
  geom_col() +
  labs(
    x = "Age (we can fix the axis labels)",
    y = "Percent Subscribed",
    fill = "Subscription Status",
    title = "Player Subscription Ratio by Age"
  ) +
  theme_minimal() +
  theme(text = element_text(size = 15))


In [ ]:
## Analyzing Correlation Between Each Set of Variables
options(repr.plot.width = 10, repr.plot.height = 6) 
all_pred |>
    ggplot(aes(x = age, y = played_hours, shape = as.factor(experience), color = subscribe)) + 
    geom_point() + 
    scale_y_log10() + 
    labs(
        x = "Age",
        y = "Hours Played",
        color = "Subscribed to Newsletter",
        shape = "Experience Level", 
        title = "Age, Hours Played, Experience and Subscription"
    ) + 
    theme(text = element_text(size = 15))
